In [14]:

import numpy as np
import os
import re
import matplotlib.pyplot as plt
import tensorflow as tf
import keras as ks
import tensorboard as tb
import PIL as P
import sklearn as sklearn
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [15]:
mobile = ks.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224, 3), 
                                                 include_top=False, 
                                                 weights='imagenet', 
                                                 pooling='avg')

In [16]:
mobile.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [17]:
dirname = os.path.join(os.getcwd(), 'fondo_de_ojo_train')
imgpath = dirname + os.sep 
 
images = []
directories = []
dircount = []
prevRoot=''
cant=0
 
print("leyendo imagenes de ",imgpath)
 
for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
 
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  /home/mirco/Escritorio/TpFinal/fondo_de_ojo_train/
/home/mirco/Escritorio/TpFinal/fondo_de_ojo_train/sano 1
/home/mirco/Escritorio/TpFinal/fondo_de_ojo_train/enfermo 815
Directorios leidos: 2
Imagenes en cada directorio [816, 826]
suma Total de imagenes en subdirs: 1642


In [18]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))
 
deportes=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice=indice+1
 
y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
 
# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  1642
0 sano
1 enfermo
Total number of outputs :  2
Output classes :  [0 1]


In [19]:
y.shape

(1642,)

In [20]:
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

Training data shape :  (1313, 224, 224, 3) (1313,)
Testing data shape :  (329, 224, 224, 3) (329,)


In [21]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Training data shape :  (1313, 224, 224, 3) (1313,)
Testing data shape :  (329, 224, 224, 3) (329,)
Original label: 0
After conversion to one-hot: [1. 0.]
(1050, 224, 224, 3) (263, 224, 224, 3) (1050, 2) (263, 2)


In [ ]:
train

In [10]:
INIT_LR = 1e-3
epochs = 6
batch_size = 64

mobile = Sequential()
mobile.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(224,224,3)))
mobile.add(LeakyReLU(alpha=0.1))
mobile.add(MaxPooling2D((2, 2),padding='same'))
mobile.add(Dropout(0.5))

mobile.add(Flatten())
mobile.add(Dense(32, activation='linear'))
mobile.add(LeakyReLU(alpha=0.1))
mobile.add(Dropout(0.5)) 
mobile.add(Dense(nClasses, activation='softmax'))

mobile.summary()

mobile.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 401408)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                12845088  
___________________________

In [ ]:

mobile_train_dropout = mobile.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
mobile.save("sports_mnist.h5py")

moile_train_dropout = mobile.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
 
# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
mobile.save("sports_mnist.h5py")

Instructions for updating:
Use tf.cast instead.
Train on 1050 samples, validate on 263 samples
Epoch 1/6


In [10]:
mobile_eval = mobile.evaluate(test_X, test_Y_one_hot, verbose=1)
 
print('Test loss:', mobile_eval[0])
print('Test accuracy:', mobile_eval[1])

329/329 [==============================] - 3s 9ms/step
Test loss: 0.7094279723326848
Test accuracy: 0.5197568396304516
